# Installing Dependencies

In [1]:
using DelimitedFiles
using NearestNeighbors: KDTree, knn
using LinearAlgebra: norm

## Using General Arrays

In [2]:
function NearNeighbor(X, t, t_knn)
    # Using kNN for Nearest Neighbors
    if t%t_knn | t == 0
        global kdtree = KDTree(X[:,1:3]')
    end 
    return kdtree
end

function force(X, r_max, s, K, kdtree)
    # Initialise displacement array
    global dX = zeros(Float64, size(X)[1], 3)

    # Loop over all cells to compute displacements
    for i in 1:size(X)[1]
        # Scan neighbours
        global idxs, _ = knn(kdtree, X[i,1:3], 14, true)

        # Initialise variables
        global Xi = X[i,1:3]
        for j in 1:size(X)[1]
            if i != j
                global r = Xi - X[j,1:3]
                global dist = norm(r)
                # Calculate attraction/repulsion force differential here
                if dist < r_max
                    global F = - K*(dist-r_max)*(dist-r_max)*(dist - s)
                    dX[i,:] +=  r/dist * F
                end
            end
        end
    end
    return dX
end

R_Agg = 16

X = readdlm("../data/Init/Two_Sphere/$R_Agg.csv", ',', Float64, header=true)[1][:, 1:3]
r_max, s = 2, 1
K = 10
t, t_knn = 0, 10
dt = 0.5

@time kd = NearNeighbor(X, t, t_knn)
@time F = force(X, r_max, s, K, kd)

  0.960368 seconds (1.46 M allocations: 86.952 MiB, 2.16% gc time)
  9.801437 seconds (112.73 M allocations: 8.330 GiB, 8.20% gc time)


6064×3 Array{Float64,2}:
  0.0         -0.00013623    0.000261888
  0.0         -0.00013623    0.000261888
  0.0         -0.00013623    0.000261888
  0.0          0.000172509   0.000337449
  0.0          0.000172509   0.000337449
  0.0          0.000172509   0.000337449
 -1.57531e-5   0.000145256   0.000337449
  3.15062e-5   0.000118003   0.000337449
  0.0          0.000118003   0.000337449
  0.0          0.000118003   0.000337449
 -1.57531e-5   9.07506e-5    0.000337449
  0.0         -3.82331e-5    4.57316e-5
  0.0         -3.82331e-5    4.57316e-5
  ⋮                         
  0.0          3.82331e-5   -4.57316e-5
  3.15062e-5  -0.000118003  -0.000337449
  0.0         -0.000118003  -0.000337449
  0.0         -0.000118003  -0.000337449
 -1.57531e-5  -9.07506e-5   -0.000337449
  0.0         -0.000172509  -0.000337449
  0.0         -0.000172509  -0.000337449
  0.0         -0.000172509  -0.000337449
 -1.57531e-5  -0.000145256  -0.000337449
  0.0          0.00013623   -0.000261888
  0.0 

## Using kNN

In [4]:
function NearNeighbor(X, t, t_knn)
    # Using kNN for Nearest Neighbors
    if t%t_knn | t == 0
        global kdtree = KDTree(X[:,1:3]')
    end 
    return kdtree
end

function force(X, r_max, s, K, kdtree)
    # Initialise displacement array
    global dX = zeros(Float64, size(X)[1], 3)

    # Loop over all cells to compute displacements
    for i in 1:size(X)[1]
        # Scan neighbours
        global idxs, _ = knn(kdtree, X[i,1:3], 14, true)

        # Initialise variables
        global Xi = X[i,1:3]
        for j in idxs
            if i != j
                global r = Xi - X[j,1:3]
                global dist = norm(r)
                # Calculate attraction/repulsion force differential here
                if dist < r_max
                    global F = - K*(dist-r_max)*(dist-r_max)*(dist - s)
                    dX[i,:] +=  r/dist * F
                end
            end
        end
    end
    return dX
end

R_Agg = 16

X = readdlm("../data/Init/Two_Sphere/$R_Agg.csv", ',', Float64, header=true)[1][:, 1:3]
r_max, s = 2, 1
K = 10
t, t_knn = 0, 10
dt = 0.5

@time kd = NearNeighbor(X, t, t_knn)
@time F = force(X, r_max, s, K, kd)

  0.016368 seconds (15.27 k allocations: 1.361 MiB)
  0.201351 seconds (1.66 M allocations: 64.323 MiB, 2.80% gc time)


6064×3 Array{Float64,2}:
  0.0         -0.00013623    0.000261888
  0.0         -0.00013623    0.000261888
  0.0         -0.00013623    0.000261888
  0.0          0.000172509   0.000337449
  0.0          0.000172509   0.000337449
  0.0          0.000172509   0.000337449
 -1.57531e-5   0.000145256   0.000337449
  3.15062e-5   0.000118003   0.000337449
  0.0          0.000118003   0.000337449
  0.0          0.000118003   0.000337449
 -1.57531e-5   9.07506e-5    0.000337449
  0.0         -3.82331e-5    4.57316e-5
  0.0         -3.82331e-5    4.57316e-5
  ⋮                         
  0.0          3.82331e-5   -4.57316e-5
  3.15062e-5  -0.000118003  -0.000337449
  0.0         -0.000118003  -0.000337449
  0.0         -0.000118003  -0.000337449
 -1.57531e-5  -9.07506e-5   -0.000337449
  0.0         -0.000172509  -0.000337449
  0.0         -0.000172509  -0.000337449
  0.0         -0.000172509  -0.000337449
 -1.57531e-5  -0.000145256  -0.000337449
  0.0          0.00013623   -0.000261888
  0.0 